# Text from Disclosures and Sections
## See list of available disclosures @ https://www.calcbench.com/disclosure_list
## Documentation @ http://calcbench.github.io/python_api_client/html/disclosures.html

In [ ]:
%pip install calcbench-api-client[BeautifulSoup, Backoff]

In [ ]:
import calcbench as cb
from tqdm.notebook import tqdm
cb.enable_backoff()

In [ ]:
tickers = cb.tickers(index="DJIA")

In [ ]:
disclosures_to_get = [
    "RiskFactors", # Calcbench does not have many conference calls transcripts
    "ManagementsDiscussionAndAnalysis",
]  # See the complete list @ https://www.calcbench.com/disclosure_list

In [ ]:
for ticker in tqdm(tickers, leave=False):
    docs = cb.document_search(
        company_identifiers=[ticker],
        disclosure_names=disclosures_to_get,
        all_history=True,
        period_type="annual",
    )
    for document in docs:
        contents = document.get_contents_text()
        print(
            f"{ticker} {document['fiscal_year']}-{document['fiscal_period']} {document['disclosure_type_name']}, Word Count {len(contents.split())}"
        )